In [15]:
import os
from transformers import AutoImageProcessor, ViTMAEForPreTraining, ViTMAEModel
from moviepy.editor import VideoFileClip
from PIL import Image
from tqdm import tqdm
import torch

In [16]:
!nvidia-smi

Mon May  6 17:32:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:81:00.0 Off |                    0 |
| N/A   31C    P0              64W / 300W |  30394MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [17]:
!ls /project/msoleyma_1026/Aff-Wild2/video_affwild2/video

1-30-1280x720.mp4	252.mp4		      463.avi
10-60-1280x720.mp4	253.mp4		      464.avi
100-29-1080x1920.mp4	254.mp4		      465.avi
101-30-1080x1920.mp4	255.mp4		      466.avi
102-30-640x360.mp4	256.mp4		      467.mp4
102.avi			257.mp4		      468.mp4
103-30-384x480.mp4	258.mp4		      469.mp4
103.avi			259.mp4		      47-30-654x480.mp4
104-17-720x480.mp4	26-60-1280x720.mp4    470.avi
105-30-1280x720.mp4	260.mp4		      471.avi
105.avi			261.mp4		      472.avi
106-30-720x1280.mp4	262.mp4		      473.avi
106.avi			264.mp4		      474.avi
107-30-640x480.mp4	265.mp4		      475.avi
107.avi			266.mp4		      476.avi
108-15-640x480.mp4	267.mp4		      477.avi
108.avi			268.mp4		      478.avi
109-30-1280x720.mp4	269.mp4		      479.avi
11-24-1920x1080.mp4	27-60-1280x720.mp4    48-30-720x1280.mp4
110-30-270x480.mp4	270.mp4		      480.avi
110.avi			271.mp4		      481.avi
111-25-1920x1080.mp4	272.mp4		      482.avi
111.avi			273.mp4		      483.mp4
112-30-640x360.mp4	274.mp4		      484.avi
112.avi			275.mp4		 

In [18]:
feature_path = '/project/msoleyma_1026/Aff-Wild2/VITMAE/feature'
feature_pooled_pre_path = '/project/msoleyma_1026/Aff-Wild2/VITMAE/feature_pooled_pre'
feature_pooled_post_path = '/project/msoleyma_1026/Aff-Wild2/VITMAE/feature_pooled_post'
video_path = '/project/msoleyma_1026/Aff-Wild2/video_affwild2/video'

In [19]:
processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-large')
# model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-large')
model = ViTMAEModel.from_pretrained("facebook/vit-mae-large", mask_ratio=0.0)
model.eval()
model.to('cuda')

ViTMAEModel(
  (embeddings): ViTMAEEmbeddings(
    (patch_embeddings): ViTMAEPatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
    )
  )
  (encoder): ViTMAEEncoder(
    (layer): ModuleList(
      (0): ViTMAELayer(
        (attention): ViTMAEAttention(
          (attention): ViTMAESelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTMAESelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTMAEIntermediate(
          (dense): Linear(in_features=1024, out_features=4096, bias=True)
          (intermediate_act_fn): GELUActivation()
        )
    

In [20]:
processor

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [21]:
model.config

ViTMAEConfig {
  "_name_or_path": "facebook/vit-mae-large",
  "architectures": [
    "ViTMAEForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.0,
  "decoder_hidden_size": 512,
  "decoder_intermediate_size": 2048,
  "decoder_num_attention_heads": 16,
  "decoder_num_hidden_layers": 8,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "mask_ratio": 0.0,
  "model_type": "vit_mae",
  "norm_pix_loss": false,
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.35.2"
}

In [22]:
import numpy as np


def extract_frames(filename, output_folder="frames"):
    """
    Extracts frames from a video iteratively and saves them as PNG images.

    Args:
        video_path: Path to the video file.
        output_folder: Folder to save the extracted frames (defaults to "frames").
    """
    file_path = os.path.join(video_path, filename)
    clip = VideoFileClip(file_path)
    features = []
    features_pooled_pre = []
    features_pooled_post = []
    # frame_count = 0
    for frame in tqdm(clip.iter_frames(), total=clip.reader.nframes):
        frame_img = Image.fromarray(frame)    # Convert frame to Pillow Image
        # print(frame_img.size)
        # print(np.array(frame_img).shape)
        inputs = processor(images=frame_img, return_tensors="pt")
        inputs.to('cuda')
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state.squeeze(0)
        # print(last_hidden_states.shape)
        features.append(last_hidden_states.detach())
        features_pooled_pre.append(last_hidden_states[1:].mean(dim=0).detach())
        features_pooled_post.append(last_hidden_states[:-1].mean(dim=0).detach())  
        # print(outputs.logits.shape)
        # frame_count += 1    
        # break
    torch.save(torch.stack(features),os.path.join(feature_path, os.path.splitext(filename)[0] + ".pt"))
    torch.save(torch.stack(features_pooled_pre),os.path.join(feature_pooled_pre_path, os.path.splitext(filename)[0] + ".pt"))
    torch.save(torch.stack(features_pooled_post),os.path.join(feature_pooled_post_path, os.path.splitext(filename)[0] + ".pt"))
    # print(torch.stack(features).shape, torch.stack(features_pooled_pre).shape, torch.stack(features_pooled_post).shape)
    clip.close()    # Close video clip
    

## Example usage
# filename = "103-30-384x480.mp4"
# extract_frames(filename)


In [23]:
480*384*3

552960

In [24]:
196*768

150528

In [27]:
file_names = [f for f in os.listdir(video_path) if os.path.isfile(os.path.join(video_path, f))]
file_names = sorted(file_names)
# file_names.reverse()
for i, filename in enumerate(file_names):
    if os.path.exists(os.path.join(feature_path, os.path.splitext(filename)[0] + ".pt")):
        continue
    if filename in ['9-15-1920x1080.mp4']:
        continue
    print(f'Starting generating feature for the {i}th video: ', filename)
    extract_frames(filename)
print("done")

Starting generating feature for the 506th video:  video17.mp4


100%|██████████| 21939/21939 [11:44<00:00, 31.13it/s]


done


In [ ]:
# class AudioVideoDataset(Dataset):
#     def __init__(self, video_dir, label_dir, device, seq_len=5, dtype=torch.float32):
#         self.video_dir = video_dir
#         self.label_dir = label_dir
#         self.device = device
#         self.seq_len = seq_len
#         self.transform = self.create_transform()
#         self.processor = Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
#         self.dtype = dtype
        
#         possible_extensions = ['.mp4', '.avi']
#         self.entries = []
#         for label_file in sorted(os.listdir(label_dir)):
#             if label_file.endswith('.txt'):
#                 base_name = os.path.splitext(label_file)[0]
#                 video_file = self.find_video_file(video_dir, base_name, possible_extensions)
#                 if video_file:
#                     self.entries.append((video_file, os.path.join(label_dir, label_file)))
#                 else:
#                     print(f"Missing video file for {label_file}")

#     def find_video_file(self, video_dir, base_name, possible_extensions):
#         for ext in possible_extensions:
#             video_path = os.path.join(video_dir, f"{base_name}{ext}")
#             if os.path.exists(video_path):
#                 return video_path
#         return None

#     def __len__(self):
#         return len(self.entries)

#     def __getitem__(self, idx):
#         video_file_path, label_file_path = self.entries[idx]
#         video, audio, info = read_video(video_file_path)
        
#         # Preprocess the audio first
#         audio_feature = self.audio_pre(audio)  # Assuming audio_pre is defined to handle this
        
#         video_fps = info['video_fps']
#         frame_interval = max(1, int(video_fps / self.seq_len))
#         audio_duration = audio_feature.shape[1] // video.shape[0]
        
#         sampled_video_frames = []
#         sampled_audio_frames = torch.empty((0,), dtype=self.dtype)  # Correct dtype
        
#         labels = torch.tensor(np.loadtxt(label_file_path, skiprows=1, delimiter=','), dtype=self.dtype)
#         sampled_labels = []

#         for i in range(0, len(video), frame_interval):
#             frame = video[i].permute(2, 0, 1)
#             sampled_video_frames.append(self.transform(frame))
#             sampled_labels.append(labels[min(i, len(labels) - 1)])  # Safeguard index
            
#             # Extract corresponding audio segment
#             start = i * audio_duration
#             end = (i + 1) * audio_duration
#             sampled_audio_segment = audio_feature[:, start:end]
#             sampled_audio_frames = torch.cat((sampled_audio_frames, sampled_audio_segment), dim=1)
        
#         video_feature = torch.stack(sampled_video_frames)
#         labels_feature = torch.stack(sampled_labels)

#          #video_feature 此时 torch.Size([num_frames, 3, 224, 224])， audio_feature还为raw， 经过hubert后，转为 torch.Size([1, num_frames, 1024])， 需要在后续模型中进行对齐

#         return video_feature, sampled_audio_frames, labels_feature
        
#     def create_transform(self):
#         #transform image to image feature
#         return transforms.Compose([
#             transforms.ToPILImage(),  # 将 numpy 数组或 tensor 转换为 PIL 图像
#             transforms.Resize((224, 224)),  # 调整图像大小
#             transforms.ToTensor(),  # 将 PIL 图像转换为 tensor，并归一化至 [0,1]
#             transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 标准化
#         ])

#     def audio_pre(self,audio):
#         """
#         input:
#                 audio preprocessed by Wav2Vec2Processor
#         return:
#                 after channel and fps adjusted audio
#         """
#         if audio.shape[0] > 1:  # More than one channel
#             audio = torch.mean(audio, dim=0, keepdim=True)

#         if info["audio_fps"] != 16000:
#         # set sample rate to int(16000 * (0.02002 * info["video_fps"])), but
#         # treat it as 16000, so that the audio speed is adjusted, in order to make
#         # the generated feature length the same as number of frames
#         # 0.02002 is a constant for HuBERT model
#             resampler = torchaudio.transforms.Resample(orig_freq=info["audio_fps"],new_freq=int(16000 * (0.02002 * info["video_fps"])))
#             resampled_audio = resampler(audio).squeeze()
        
#         audio_feature = self.processor(resampled_audio, return_tensors="pt", sampling_rate=16000).input_values
#         audio_feature = audio_feature.to(device, dtype = self.dtype)

#         return audio_feature

In [ ]:
# ViTMAEModel(
#   (embeddings): ViTMAEEmbeddings(
#     (patch_embeddings): ViTMAEPatchEmbeddings(
#       (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
#     )
#   )
#   (encoder): ViTMAEEncoder(
#     (layer): ModuleList(
#       (0): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (1): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (2): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (3): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (4): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (5): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (6): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (7): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (8): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (9): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (10): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (11): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (12): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (13): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (14): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (15): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (16): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (17): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (18): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (19): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (20): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (21): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (22): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#       (23): ViTMAELayer(
#         (attention): ViTMAEAttention(
#           (attention): ViTMAESelfAttention(
#             (query): Linear(in_features=1024, out_features=1024, bias=True)
#             (key): Linear(in_features=1024, out_features=1024, bias=True)
#             (value): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#           (output): ViTMAESelfOutput(
#             (dense): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#           )
#         )
#         (intermediate): ViTMAEIntermediate(
#           (dense): Linear(in_features=1024, out_features=4096, bias=True)
#           (intermediate_act_fn): GELUActivation()
#         )
#         (output): ViTMAEOutput(
#           (dense): Linear(in_features=4096, out_features=1024, bias=True)
#           (dropout): Dropout(p=0.0, inplace=False)
#         )
#         (layernorm_before): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#         (layernorm_after): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
#       )
#     )
#   )
#   (layernorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
# )